## Databases and Data Warehouses Assignment - 7

**Overview** :
This assignment focuses on performing an end-to-end ETL (Extract, Transform, Load) process using the Titanic dataset from Kaggle. The ETL process is integrated with a simple machine-learning pipeline to demonstrate practical applications in a data science context.

### Team Members (Group – 08)

##### 1.Rutika Rajesh Bankar - 25PGAI0103

##### 2.Rishabh Gaur - 25PGAI0023

##### 3.Mukesh Kumar Khemani - 25PGAI0115

##### 4.Guna Shekhar Dasyam - 25PGAI0063

##### 5.Nagendra Jupudy - 25PGAI0146



#### Part 1: Data Extraction
**Objective:** 
#### Extract the Titanic dataset from Kaggle and load it into a pandas DataFrame.

**Implementation:**

##### - Utilized the Kaggle API to download the dataset.
##### - Loaded the dataset using pandas' read_csv function.'

In [1]:
import pandas as pd
data = pd.read_csv('C:\\Users\\india\\Desktop\\Programming_with_python\\Assignment_7_Database\\titanic\\train.csv')
# reading the train.csv file from the given path

In [2]:
print(data.head())
# printing the first 5 rows of the data

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S  


In [3]:
data['Age'].fillna(data['Age'].median(), inplace=True)
# filling the missing values in the Age column with the median of the Age column

C:\Users\india\AppData\Local\Temp\ipykernel_12528\913469570.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Age'].fillna(data['Age'].median(), inplace=True)


In [4]:
data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)
# filling the missing values in the Embarked column with the mode of the Embarked column

C:\Users\india\AppData\Local\Temp\ipykernel_12528\4247733614.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)


In [5]:
data.drop(columns=['Cabin'], inplace=True)
# dropping the Cabin column


#### Part 2: Data Transformation
**Objective** :
Handle missing values, perform feature engineering, and prepare data by scaling numerical features and encoding categorical features for machine learning.

**Steps and Tasks:**

**Handling Missing Values:**

##### - Filled missing values in 'Age' with the median.
##### - Replaced missing values in 'Embarked' with the most common embarkation point.
##### - Dropped the 'Cabin' column due to high missing values.

**Feature Engineering**:

##### - Extracted titles from passenger names.
##### - Created a new feature 'FamilySize' based on the number of siblings/spouses and parents/children aboard.

**Data Scaling and Encoding**:

##### - Scaled numerical features (Age, Fare, FamilySize) using StandardScaler.
##### - Encoded categorical features (Sex, Embarked, Title) using OneHotEncoder.

In [6]:
data['Title'] = data['Name'].apply(lambda x: x.split(',')[1].split('.')[0].strip())
# creating a new column Title by extracting the title from the Name column

In [7]:
data['FamilySize'] = data['SibSp'] + data['Parch'] + 1
# creating a new column FamilySize by adding the SibSp and Parch columns

In [8]:
numerical_cols = ['Age', 'Fare', 'FamilySize']
categorical_cols = ['Sex', 'Embarked', 'Title']
# defining the numerical and categorical columns

In [9]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
# importing the necessary libraries

In [10]:
numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown='ignore')
# defining the numerical and categorical transformers

In [11]:
preprocessor = ColumnTransformer(
transformers=[
('num', numerical_transformer, numerical_cols),
('cat', categorical_transformer, categorical_cols)
])
# defining the preprocessor

In [12]:
data_transformed = preprocessor.fit_transform(data)
# transforming the data

In [13]:
numerical_col_names = numerical_cols
categorical_col_names = list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_cols))
transformed_columns = numerical_col_names + categorical_col_names
# getting the transformed columns

In [32]:
print("Shape of data_transformed:", data_transformed.shape)
# printing the shape of the transformed data

Shape of data_transformed: (891, 25)


In [15]:
print("Transformed columns:", transformed_columns)
print("Length of transformed_columns:", len(transformed_columns))
# printing the transformed columns and the length of the transformed columns

Transformed columns: ['Age', 'Fare', 'FamilySize', 'Sex_female', 'Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S', 'Title_Capt', 'Title_Col', 'Title_Don', 'Title_Dr', 'Title_Jonkheer', 'Title_Lady', 'Title_Major', 'Title_Master', 'Title_Miss', 'Title_Mlle', 'Title_Mme', 'Title_Mr', 'Title_Mrs', 'Title_Ms', 'Title_Rev', 'Title_Sir', 'Title_the Countess']
Length of transformed_columns: 25


In [16]:
if data_transformed.shape[1] != len(transformed_columns):
    raise ValueError("Mismatch between number of columns in data_transformed and number of feature names.")
# checking if the number of columns in data_transformed and the number of feature names are equal

In [17]:
data_transformed_df = pd.DataFrame(data = data_transformed.todense(), columns=transformed_columns)
# creating a DataFrame from the transformed data

In [18]:
data_transformed_df.shape
# printing the shape of the transformed DataFrame

(891, 25)

#### Part 3: Data Loading
**Objective**:
Store the cleaned and transformed data in an SQLite database and demonstrate data retrieval.

**Implementation**:

Utilized SQLAlchemy to create an SQLite database and stored the transformed data.
Implemented a function to retrieve data from the database to verify successful storage.

In [19]:
from sqlalchemy import create_engine
# importing the necessary library

In [20]:
# !pip install sqlalchemy
# installing the sqlalchemy library (if not installed)

In [21]:
engine = create_engine('sqlite:///titanic.db')
# creating a database engine

In [22]:
data_transformed_df.to_sql('titanic_transformed', engine, index=False, if_exists='replace')
# writing the transformed DataFrame to the database

891

In [23]:
# Function to load data from the database
def load_data_from_db(engine):
    query = "SELECT * FROM titanic_transformed"
    data_from_db = pd.read_sql(query, engine)
    return data_from_db

In [24]:
# Load the data
data_loaded = load_data_from_db(engine)
print(data_loaded.head())
# printing the first 5 rows of the data loaded from the database

        Age      Fare  FamilySize  Sex_female  Sex_male  Embarked_C  \
0 -0.565736 -0.502445    0.059160         0.0       1.0         0.0   
1  0.663861  0.786845    0.059160         1.0       0.0         1.0   
2 -0.258337 -0.488854   -0.560975         1.0       0.0         0.0   
3  0.433312  0.420730    0.059160         1.0       0.0         0.0   
4  0.433312 -0.486337   -0.560975         0.0       1.0         0.0   

   Embarked_Q  Embarked_S  Title_Capt  Title_Col  ...  Title_Master  \
0         0.0         1.0         0.0        0.0  ...           0.0   
1         0.0         0.0         0.0        0.0  ...           0.0   
2         0.0         1.0         0.0        0.0  ...           0.0   
3         0.0         1.0         0.0        0.0  ...           0.0   
4         0.0         1.0         0.0        0.0  ...           0.0   

   Title_Miss  Title_Mlle  Title_Mme  Title_Mr  Title_Mrs  Title_Ms  \
0         0.0         0.0        0.0       1.0        0.0       0.0   
1   

#### Part 4: Integration with ML Pipeline
**Objective**:
Build and evaluate a logistic regression model using the transformed and loaded data.

**Implementation**:

Split data into training and testing sets.
Trained a logistic regression model and evaluated its accuracy on the testing set.

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
# importing the necessary libraries

In [26]:
# Define the features and target variable
X = data_loaded
y = data['Survived']


In [27]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [28]:
# Train a logistic regression model
model = LogisticRegression(max_iter=1000)


In [29]:
model.fit(X_train, y_train)
# Fit the model

LogisticRegression(max_iter=1000)

In [30]:
# Make predictions
y_pred = model.predict(X_test)

In [31]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

Model Accuracy: 0.82


**Documentation and Code Quality**
##### - Detailed comments and clear code structure are maintained throughout the scripts to ensure readability and maintainability.
##### - Included a README.md file with environment setup and script execution instructions.


**Conclusion**:
#### This assignment effectively demonstrates the capability to perform an ETL job integrated with a machine learning pipeline, highlighting practical data science skills from data extraction to model evaluation.